In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model, metrics, ensemble
from sklearn import model_selection
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, GridSearchCV
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
def read_file(filename):
    raw_data = pd.read_csv(filename)
    data = raw_data.copy()
    return data

In [3]:
# read in train and test data -> X_train, y_train, X_test (pd.df)
train_path = "train_2008.csv"
test_path = "test_2008.csv"
train_data = read_file(train_path)
test_data = read_file(test_path)

y_train = train_data['target']
X_train = train_data.drop('target', axis=1)
X_train = X_train.drop('id', axis=1)

X_test = test_data
X_test = X_test.drop('id', axis=1)

In [4]:
# drop same_resp feature
def select_same_resp_feature(X_train):
    same_resp_feature = []
    for feature in X_train.columns:
        if len(np.unique(X_train[feature])) == 1:
            same_resp_feature.append(feature)
    return same_resp_feature

same_resp_feature = select_same_resp_feature(X_train)
X_train.drop(same_resp_feature, axis=1, inplace=True)
X_test.drop(same_resp_feature, axis=1, inplace=True)
print(len(same_resp_feature))
print(same_resp_feature)

15
['HRMONTH', 'HRYEAR4', 'HUTYPEA', 'HUTYPC', 'HRINTSTA', 'PEAFNOW', 'PRPERTYP', 'PULKDK4', 'PULKDK5', 'PULKDK6', 'PULKPS4', 'PULKPS5', 'PULKPS6', 'HXPHONEO', 'PXAGE']


In [5]:
# drop low resp features
response_rates = X_train[X_train >= 0].count() / len(X_train)
mostly_blank_feats = []
for itm in response_rates.items():
    if itm[1] < 0.01:
#         X_train[feature] = X_train[feature].apply(lambda x: np.NaN if (x == -1) else x)
#         X_test[feature] = X_test[feature].apply(lambda x: np.NaN if (x == -1) else x)
        mostly_blank_feats.append(itm[0])
print(len(mostly_blank_feats))

X_train.drop(mostly_blank_feats, axis=1, inplace=True)
X_test.drop(mostly_blank_feats, axis=1, inplace=True)

58


In [6]:
# encode
# s = X_train['HUFINAL']
# out = s.value_counts(normalize=True)
# print(out)
# print(type(out))
# out.max() > 0.9
# maxx = out.idxmax()
# X_train['HUFINAL']= X_train['HUFINAL'].apply(lambda x: 1 if (x == maxx) else 0)
# print(X_train['HUFINAL'].values[:10])

# recode
for i in X_train.columns:
    cols = X_train[i]
    print(type(cols))
    #print(cols)
    freq_out = cols.value_counts(normalize=True)
    if freq_out.max() > 0.95:
        max_idx = freq_out.idxmax()
        X_train[i] = X_train[i].apply(lambda x: 1 if (x == max_idx) else 0)
        X_test[i] = X_test[i].apply(lambda x: 1 if (x == max_idx) else 0)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.S

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.S

In [ ]:
print(X_train.values.shape)

In [9]:
print(X_train.values[:10, :10])

[[16000     1   201     1     1     1     1     1     1    16]
 [16001     1   201     1     2     1     1     1     1     6]
 [16002     1     1     1     1     1     1     1     1    15]
 [16003     2     1     1     1     1     1     1     1    13]
 [16004     2   201     1     1     1     1     1     1    -3]
 [16005     2   201     1     1     0     1     1     1    13]
 [16006     2   201     1     1     1     1     1     1    -3]
 [16007     1   201     1     2     1     0     0     1     7]
 [16008     1   201     1     3     0     1     1     1     1]
 [16009     1   201     1     1     1     1     1     1    13]]


In [5]:
# negative -> -1
for feature in X_train.columns:
    X_train[feature] = X_train[feature].apply(lambda x: np.NaN if (x == -1) else x)
    X_test[feature] = X_test[feature].apply(lambda x: np.NaN if (x == -1) else x)

In [6]:
# encode some features
categorical_features = ['HUFINAL','HEHOUSUT','GEREG', 'HUBUS', 'PTDTRACE', 'PENATVTY', 'PUABSOT', 'PEIO1COW', 
                     'HUFINAL', 'GESTCEN', 'GESTFIPS', #'PEIO1ICD', 'PEIO2ICD', 
                     'PRCITSHP', 'PUDIS', 'PRABSREA', 'PRWKSTAT', 'HUPRSCNT', 
                     'PERRP', 'GTCBSAST', 'PRMJOCGR', 'HRHTYPE', ]
# Now dummy these features
train_dummy_df = pd.DataFrame()
test_dummy_df = pd.DataFrame()

new_cat = []
for feature in categorical_features:
    if feature in X_train.columns:
        new_cat.append(feature)
        train_dummy_vars = pd.get_dummies(X_train[feature], prefix=feature)
        train_dummy_df = pd.concat([train_dummy_df, train_dummy_vars], axis=1)

        test_dummy_vars = pd.get_dummies(X_test[feature], prefix=feature)
        test_dummy_df = pd.concat([test_dummy_df, test_dummy_vars], axis=1)
# Drop the original categorical variables
X_train.drop(new_cat, axis=1, inplace=True)
X_test.drop(new_cat, axis=1, inplace=True)
# Add dummy vars to the data
X_train = pd.concat([X_train, train_dummy_df], axis=1)
X_test = pd.concat([X_test, test_dummy_df], axis=1)

# train_test 
feats_to_add_to_train = [f for f in X_test.columns if f not in X_train.columns]
feats_to_add_to_test = [f for f in X_train.columns if f not in X_test.columns]
for feat in feats_to_add_to_train:
    X_train[feat] = np.nan
for feat in feats_to_add_to_test:
    X_test[feat] = np.nan

In [6]:
    
X = X_train.values
Y = y_train.values
X_t = X_test.values
print(X.shape)
print(Y.shape)
print(X_t.shape)

(64667, 366)
(64667,)
(16000, 366)


In [30]:
print(X[:10, :10])

[[1.6000e+04 1.0000e+00 2.0100e+02 0.0000e+00 1.0000e+00 1.0000e+00
  1.0000e+00        nan 1.0000e+00 1.6000e+01]
 [1.6001e+04 1.0000e+00 2.0100e+02 0.0000e+00 2.0000e+00 1.0000e+00
  1.0000e+00        nan 1.0000e+00 6.0000e+00]
 [1.6002e+04 1.0000e+00 1.0000e+00 0.0000e+00 1.0000e+00 1.0000e+00
  1.0000e+00        nan 1.0000e+00 1.5000e+01]
 [1.6003e+04 2.0000e+00 1.0000e+00 0.0000e+00 1.0000e+00 1.0000e+00
  1.0000e+00        nan 1.0000e+00 1.3000e+01]
 [1.6004e+04 2.0000e+00 2.0100e+02 0.0000e+00 1.0000e+00 1.0000e+00
  1.0000e+00        nan 1.0000e+00        nan]
 [1.6005e+04 2.0000e+00 2.0100e+02 0.0000e+00 1.0000e+00 6.0000e+00
  1.0000e+00        nan 1.0000e+00 1.3000e+01]
 [1.6006e+04 2.0000e+00 2.0100e+02 0.0000e+00 1.0000e+00 1.0000e+00
  1.0000e+00        nan 1.0000e+00        nan]
 [1.6007e+04 1.0000e+00 2.0100e+02 0.0000e+00 2.0000e+00 1.0000e+00
  2.0000e+00 2.0000e+00 1.0000e+00 7.0000e+00]
 [1.6008e+04 1.0000e+00 2.0100e+02 0.0000e+00 3.0000e+00 5.0000e+00
  1.0000e+00

In [8]:
test_size = 0.3
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=10)
clf_xgb = XGBClassifier(max_depth=9, gamma=0.25, subsample=0.7, min_child_weight=2, n_estimators=50, objective='binary:logistic') 
clf_xgb.fit(X_train, y_train)
y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

train_auc = metrics.roc_auc_score(y_train, y_train_pred)
print("train auc: ", train_auc)
test_auc = metrics.roc_auc_score(y_test, y_test_pred)
print("test auc: ", test_auc)

train auc:  0.9012511564813253
test auc:  0.7906480822504987


In [11]:
y_test_pred_prob = clf_xgb.predict_proba(X_t)[:, 1]
pd.DataFrame(y_test_pred_prob).to_csv("xgb_test_po2008.csv")

In [ ]:
# def cross_validation(num_folds, X_train_arr, clf):
#     kf = model_selection.KFold(n_splits=num_folds)
#     auc = []
#     for train_index, test_index in kf.split(X_train_arr):
#         print("1---")
#         X_train_cv, X_test_cv = X_train_arr[train_index], X_train_arr[test_index]
#         y_train_cv, y_test_cv = y_train_arr[train_index], y_train_arr[test_index]
#         clf.fit(X_train_cv,y_train_cv)
#         y_pred_prob = clf.predict_proba(X_test_cv)[:, 1]
#         auc.append(metrics.roc_auc_score(y_test_cv, y_pred_prob))
#         print("2---")
#     return auc

In [7]:
# 1
kf =KFold(n_splits=4)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf_xgb = XGBClassifier(max_depth=9, gamma=0.25, subsample=0.7, min_child_weight=2, n_estimators=50, objective='binary:logistic', scale_pos_weight=3) 
    clf_xgb.fit(X_train, y_train)
    y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
    y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

    train_auc = metrics.roc_auc_score(y_train, y_train_pred)
    print("train auc: ", train_auc)
    test_auc = metrics.roc_auc_score(y_test, y_test_pred)
    print("test auc: ", test_auc)
print(len(X_train))
print(len(X_test))

TRAIN: [16167 16168 16169 ... 64664 64665 64666] TEST: [    0     1     2 ... 16164 16165 16166]
train auc:  0.9105435504991333
test auc:  0.7811150764551923
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [16167 16168 16169 ... 32331 32332 32333]
train auc:  0.9113135473798598
test auc:  0.7781658839485525
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [32334 32335 32336 ... 48498 48499 48500]
train auc:  0.9146942468814164
test auc:  0.7854101715814161
TRAIN: [    0     1     2 ... 48498 48499 48500] TEST: [48501 48502 48503 ... 64664 64665 64666]
train auc:  0.9123738900267131
test auc:  0.7778787695298433
48501
16166


In [9]:
# 1 no id
kf =KFold(n_splits=4)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf_xgb = XGBClassifier(max_depth=9, gamma=0.25, subsample=0.7, min_child_weight=2, n_estimators=50, objective='binary:logistic', scale_pos_weight=3) 
    clf_xgb.fit(X_train, y_train)
    y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
    y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

    train_auc = metrics.roc_auc_score(y_train, y_train_pred)
    print("train auc: ", train_auc)
    test_auc = metrics.roc_auc_score(y_test, y_test_pred)
    print("test auc: ", test_auc)
print(len(X_train))
print(len(X_test))

TRAIN: [16167 16168 16169 ... 64664 64665 64666] TEST: [    0     1     2 ... 16164 16165 16166]
train auc:  0.909734721805481
test auc:  0.7863631796376646
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [16167 16168 16169 ... 32331 32332 32333]
train auc:  0.9090972792165517
test auc:  0.7808630791380613
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [32334 32335 32336 ... 48498 48499 48500]
train auc:  0.9089077116642704
test auc:  0.7893985086199315
TRAIN: [    0     1     2 ... 48498 48499 48500] TEST: [48501 48502 48503 ... 64664 64665 64666]
train auc:  0.9129444580402374
test auc:  0.7806461717589306
48501
16166


In [8]:
# 2
kf =KFold(n_splits=4)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf_xgb = XGBClassifier(max_depth=9, gamma=0.3, subsample=0.8, min_child_weight=2, n_estimators=50, colsample_bytree = 0.8, objective='binary:logistic') 
    clf_xgb.fit(X_train, y_train)
    y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
    y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

    train_auc = metrics.roc_auc_score(y_train, y_train_pred)
    print("train auc: ", train_auc)
    test_auc = metrics.roc_auc_score(y_test, y_test_pred)
    print("test auc: ", test_auc)
print(len(X_train))
print(len(X_test))

TRAIN: [16167 16168 16169 ... 64664 64665 64666] TEST: [    0     1     2 ... 16164 16165 16166]
train auc:  0.9035597315654079
test auc:  0.7850145914019138
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [16167 16168 16169 ... 32331 32332 32333]
train auc:  0.9047741891234785
test auc:  0.7809926933625121
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [32334 32335 32336 ... 48498 48499 48500]
train auc:  0.9067856950917265
test auc:  0.789128436590709
TRAIN: [    0     1     2 ... 48498 48499 48500] TEST: [48501 48502 48503 ... 64664 64665 64666]
train auc:  0.908394412438909
test auc:  0.7826559379667513
48501
16166


In [9]:
# 3
kf =KFold(n_splits=4)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf_xgb = XGBClassifier(max_depth=9, gamma=0.3, subsample=0.8, min_child_weight=3, n_estimators=100, objective='binary:logistic') 
    clf_xgb.fit(X_train, y_train)
    y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
    y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

    train_auc = metrics.roc_auc_score(y_train, y_train_pred)
    print("train auc: ", train_auc)
    test_auc = metrics.roc_auc_score(y_test, y_test_pred)
    print("test auc: ", test_auc)
print(len(X_train))
print(len(X_test))

TRAIN: [16167 16168 16169 ... 64664 64665 64666] TEST: [    0     1     2 ... 16164 16165 16166]
train auc:  0.927976692420224
test auc:  0.7865624805836302
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [16167 16168 16169 ... 32331 32332 32333]
train auc:  0.9299278019067931
test auc:  0.785509121523048
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [32334 32335 32336 ... 48498 48499 48500]
train auc:  0.9330839373409316
test auc:  0.7936869523973387
TRAIN: [    0     1     2 ... 48498 48499 48500] TEST: [48501 48502 48503 ... 64664 64665 64666]
train auc:  0.9325190206537745
test auc:  0.7844562077969776
48501
16166


In [10]:
# 3 no id
kf =KFold(n_splits=4)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf_xgb = XGBClassifier(max_depth=9, gamma=0.3, subsample=0.8, min_child_weight=3, n_estimators=100, objective='binary:logistic') 
    clf_xgb.fit(X_train, y_train)
    y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
    y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

    train_auc = metrics.roc_auc_score(y_train, y_train_pred)
    print("train auc: ", train_auc)
    test_auc = metrics.roc_auc_score(y_test, y_test_pred)
    print("test auc: ", test_auc)
print(len(X_train))
print(len(X_test))

TRAIN: [16167 16168 16169 ... 64664 64665 64666] TEST: [    0     1     2 ... 16164 16165 16166]
train auc:  0.9270746279799869
test auc:  0.7903525660836146
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [16167 16168 16169 ... 32331 32332 32333]
train auc:  0.928477150420264
test auc:  0.7841588565682915
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [32334 32335 32336 ... 48498 48499 48500]
train auc:  0.9287558459798065
test auc:  0.7941368320882467
TRAIN: [    0     1     2 ... 48498 48499 48500] TEST: [48501 48502 48503 ... 64664 64665 64666]
train auc:  0.9293163019484469
test auc:  0.7857186376779245
48501
16166


In [15]:
clf_xgb = XGBClassifier(max_depth=9, gamma=0.3, subsample=0.8, min_child_weight=3, n_estimators=100, objective='binary:logistic') 
clf_xgb.fit(X, Y)
y_test_pred_prob = clf_xgb.predict_proba(X_t)[:, 1]
pd.DataFrame(y_test_pred_prob).to_csv("xgboost_143_2.csv")

In [11]:
# 4
kf =KFold(n_splits=4)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf_xgb = XGBClassifier(max_depth=9, gamma=0.3, subsample=0.8, min_child_weight=3, n_estimators=100, colsample_bytree = 0.8, objective='binary:logistic') 
    clf_xgb.fit(X_train, y_train)
    y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
    y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

    train_auc = metrics.roc_auc_score(y_train, y_train_pred)
    print("train auc: ", train_auc)
    test_auc = metrics.roc_auc_score(y_test, y_test_pred)
    print("test auc: ", test_auc)
print(len(X_train))
print(len(X_test))

TRAIN: [16167 16168 16169 ... 64664 64665 64666] TEST: [    0     1     2 ... 16164 16165 16166]
train auc:  0.9316039932331532
test auc:  0.7861549694095094
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [16167 16168 16169 ... 32331 32332 32333]
train auc:  0.9286497485035727
test auc:  0.7829393208210548
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [32334 32335 32336 ... 48498 48499 48500]
train auc:  0.9314564411820098
test auc:  0.7909384409632179
TRAIN: [    0     1     2 ... 48498 48499 48500] TEST: [48501 48502 48503 ... 64664 64665 64666]
train auc:  0.9327205934048538
test auc:  0.785250431973734
48501
16166


In [12]:
# 5 no id
kf =KFold(n_splits=4)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf_xgb = XGBClassifier(max_depth=8, gamma=0.3, subsample=0.8, min_child_weight=4, n_estimators=150, colsample_bytree = 0.8, objective='binary:logistic') 
    clf_xgb.fit(X_train, y_train)
    y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
    y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

    train_auc = metrics.roc_auc_score(y_train, y_train_pred)
    print("train auc: ", train_auc)
    test_auc = metrics.roc_auc_score(y_test, y_test_pred)
    print("test auc: ", test_auc)
print(len(X_train))
print(len(X_test))

TRAIN: [16167 16168 16169 ... 64664 64665 64666] TEST: [    0     1     2 ... 16164 16165 16166]
train auc:  0.9160847349442498
test auc:  0.7904776600912051
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [16167 16168 16169 ... 32331 32332 32333]
train auc:  0.9141984175754483
test auc:  0.787550737355693
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [32334 32335 32336 ... 48498 48499 48500]
train auc:  0.9136903351115206
test auc:  0.7937981579379064
TRAIN: [    0     1     2 ... 48498 48499 48500] TEST: [48501 48502 48503 ... 64664 64665 64666]
train auc:  0.9140249323685181
test auc:  0.7885771244282236
48501
16166


In [13]:
clf_xgb = XGBClassifier(max_depth=8, gamma=0.3, subsample=0.8, min_child_weight=4, n_estimators=150, colsample_bytree = 0.8, objective='binary:logistic') 
clf_xgb.fit(X, Y)
y_test_pred_prob = clf_xgb.predict_proba(X_t)[:, 1]
pd.DataFrame(y_test_pred_prob).to_csv("xgboost_155.csv")

In [18]:
test12_path = "test_2012.csv"
test12_data = read_file(test12_path)
X_test12 = test12_data
X_test12 = X_test12.drop('id', axis=1)
X_test12.drop(same_resp_feature, axis=1, inplace=True)
for feature in X_test12.columns:
    X_test12[feature] = X_test12[feature].apply(lambda x: np.NaN if (x == -1) else x)
X_t12 = X_test12.values
print(X_t12.shape)
y_test12_pred_prob = clf_xgb.predict_proba(X_t12)[:, 1]
pd.DataFrame(y_test12_pred_prob).to_csv("xgboost12_155.csv")

(82820, 366)


In [7]:
# 5 no id train on 3rd
kf =KFold(n_splits=4)
# print(kf.split(X))
index = []
in_test = []
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    index.append(train_index)
    in_test.append(test_index)
#     y_k.append(y_train)
#     clf_xgb = XGBClassifier(max_depth=8, gamma=0.3, subsample=0.8, min_child_weight=4, n_estimators=150, colsample_bytree = 0.8, objective='binary:logistic') 
#     clf_xgb.fit(X_train, y_train)
#     y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
#     y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

#     train_auc = metrics.roc_auc_score(y_train, y_train_pred)
#     print("train auc: ", train_auc)
#     test_auc = metrics.roc_auc_score(y_test, y_test_pred)
#     print("test auc: ", test_auc)
# print(len(X_train))
# print(len(X_test))
# print(X_k)
# print(index)
X_train, X_test = X[index[2]], X[in_test[2]]
y_train, y_test = Y[index[2]], Y[in_test[2]]
clf_xgb = XGBClassifier(max_depth=8, gamma=0.3, subsample=0.8, min_child_weight=4, n_estimators=150, colsample_bytree = 0.8, objective='binary:logistic') 
clf_xgb.fit(X_train, y_train)
y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

TRAIN: [16167 16168 16169 ... 64664 64665 64666] TEST: [    0     1     2 ... 16164 16165 16166]
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [16167 16168 16169 ... 32331 32332 32333]
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [32334 32335 32336 ... 48498 48499 48500]
TRAIN: [    0     1     2 ... 48498 48499 48500] TEST: [48501 48502 48503 ... 64664 64665 64666]


In [8]:
train_auc = metrics.roc_auc_score(y_train, y_train_pred)
print("train auc: ", train_auc)
test_auc = metrics.roc_auc_score(y_test, y_test_pred)
print("test auc: ", test_auc)

train auc:  0.9136903351115206
test auc:  0.7937981579379064


In [9]:
y_test_pred_prob = clf_xgb.predict_proba(X_t)[:, 1]
pd.DataFrame(y_test_pred_prob).to_csv("xgboost_1304.csv")

In [6]:
# 5 no id no nan
kf =KFold(n_splits=4)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf_xgb = XGBClassifier(max_depth=8, gamma=0.3, subsample=0.8, min_child_weight=4, n_estimators=150, colsample_bytree = 0.8, objective='binary:logistic') 
    clf_xgb.fit(X_train, y_train)
    y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
    y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

    train_auc = metrics.roc_auc_score(y_train, y_train_pred)
    print("train auc: ", train_auc)
    test_auc = metrics.roc_auc_score(y_test, y_test_pred)
    print("test auc: ", test_auc)
print(len(X_train))
print(len(X_test))

TRAIN: [16167 16168 16169 ... 64664 64665 64666] TEST: [    0     1     2 ... 16164 16165 16166]
train auc:  0.917018773992156
test auc:  0.7929736040924529
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [16167 16168 16169 ... 32331 32332 32333]
train auc:  0.9162815892464278
test auc:  0.7894076240209988
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [32334 32335 32336 ... 48498 48499 48500]
train auc:  0.912373269320724
test auc:  0.7939943600043025
TRAIN: [    0     1     2 ... 48498 48499 48500] TEST: [48501 48502 48503 ... 64664 64665 64666]
train auc:  0.9144277057681898
test auc:  0.7906144554281467
48501
16166


In [6]:
clf_xgb = XGBClassifier(max_depth=8, gamma=0.3, subsample=0.8, min_child_weight=4, n_estimators=150, colsample_bytree = 0.8, objective='binary:logistic') 
clf_xgb.fit(X, Y)
# y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
# y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

# train_auc = metrics.roc_auc_score(y_train, y_train_pred)
# print("train auc: ", train_auc)
# test_auc = metrics.roc_auc_score(y_test, y_test_pred)
# print("test auc: ", test_auc)
y_test_pred_prob = clf_xgb.predict_proba(X_t)[:, 1]
pd.DataFrame(y_test_pred_prob).to_csv("xgboost_1038.csv")

In [7]:
test12_path = "test_2012.csv"
test12_data = read_file(test12_path)
X_test12 = test12_data
X_test12 = X_test12.drop('id', axis=1)
X_test12.drop(same_resp_feature, axis=1, inplace=True)
# for feature in X_test12.columns:
#     X_test12[feature] = X_test12[feature].apply(lambda x: np.NaN if (x == -1) else x)
X_t12 = X_test12.values
print(X_t12.shape)
y_test12_pred_prob = clf_xgb.predict_proba(X_t12)[:, 1]
pd.DataFrame(y_test12_pred_prob).to_csv("xgboost12_1038.csv")

(82820, 366)


In [8]:
# 5 no id    nan + one hot
kf =KFold(n_splits=4)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf_xgb = XGBClassifier(max_depth=8, gamma=0.3, subsample=0.8, min_child_weight=4, n_estimators=150, colsample_bytree = 0.8, objective='binary:logistic') 
    clf_xgb.fit(X_train, y_train)
    y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
    y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

    train_auc = metrics.roc_auc_score(y_train, y_train_pred)
    print("train auc: ", train_auc)
    test_auc = metrics.roc_auc_score(y_test, y_test_pred)
    print("test auc: ", test_auc)
print(len(X_train))
print(len(X_test))

TRAIN: [16167 16168 16169 ... 64664 64665 64666] TEST: [    0     1     2 ... 16164 16165 16166]
train auc:  0.9065732794772703
test auc:  0.7919557615812174
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [16167 16168 16169 ... 32331 32332 32333]
train auc:  0.9067079304704138
test auc:  0.7883497199019323
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [32334 32335 32336 ... 48498 48499 48500]
train auc:  0.9059213288265974
test auc:  0.793846249527046
TRAIN: [    0     1     2 ... 48498 48499 48500] TEST: [48501 48502 48503 ... 64664 64665 64666]
train auc:  0.9057924998519122
test auc:  0.7878698688525368
48501
16166


In [9]:
clf_xgb = XGBClassifier(max_depth=8, gamma=0.3, subsample=0.8, min_child_weight=4, n_estimators=150, colsample_bytree = 0.8, objective='binary:logistic') 
clf_xgb.fit(X, Y)
y_test_pred_prob = clf_xgb.predict_proba(X_t)[:, 1]
pd.DataFrame(y_test_pred_prob).to_csv("xgboost_1142.csv")

In [19]:
# 6 no id
kf =KFold(n_splits=4)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf_xgb = XGBClassifier(max_depth=8, gamma=0.3, subsample=0.8, min_child_weight=4, n_estimators=150, colsample_bytree = 0.8, objective='binary:logistic', scale_pos_weight = 3) 
    clf_xgb.fit(X_train, y_train)
    y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
    y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

    train_auc = metrics.roc_auc_score(y_train, y_train_pred)
    print("train auc: ", train_auc)
    test_auc = metrics.roc_auc_score(y_test, y_test_pred)
    print("test auc: ", test_auc)
print(len(X_train))
print(len(X_test))

TRAIN: [16167 16168 16169 ... 64664 64665 64666] TEST: [    0     1     2 ... 16164 16165 16166]
train auc:  0.928662501168097
test auc:  0.7919047632723833
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [16167 16168 16169 ... 32331 32332 32333]
train auc:  0.9268014796121307
test auc:  0.7832425450697638
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [32334 32335 32336 ... 48498 48499 48500]
train auc:  0.9259854388440252
test auc:  0.7903221635726942
TRAIN: [    0     1     2 ... 48498 48499 48500] TEST: [48501 48502 48503 ... 64664 64665 64666]
train auc:  0.9298428966662761
test auc:  0.7862417750855438
48501
16166


In [20]:
# 7 no id
kf =KFold(n_splits=4)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf_xgb = XGBClassifier(max_depth=9, gamma=0.3, subsample=0.8, min_child_weight=4, n_estimators=150, colsample_bytree = 0.8, objective='binary:logistic') 
    clf_xgb.fit(X_train, y_train)
    y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
    y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

    train_auc = metrics.roc_auc_score(y_train, y_train_pred)
    print("train auc: ", train_auc)
    test_auc = metrics.roc_auc_score(y_test, y_test_pred)
    print("test auc: ", test_auc)
print(len(X_train))
print(len(X_test))

TRAIN: [16167 16168 16169 ... 64664 64665 64666] TEST: [    0     1     2 ... 16164 16165 16166]
train auc:  0.9425954417601865
test auc:  0.7913899260915883
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [16167 16168 16169 ... 32331 32332 32333]
train auc:  0.9395710495653548
test auc:  0.7853460782649373
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [32334 32335 32336 ... 48498 48499 48500]
train auc:  0.938112298525706
test auc:  0.7949214200258504
TRAIN: [    0     1     2 ... 48498 48499 48500] TEST: [48501 48502 48503 ... 64664 64665 64666]
train auc:  0.9381091050717552
test auc:  0.7869330418945263
48501
16166


In [21]:
# 8 no id
kf =KFold(n_splits=4)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf_xgb = XGBClassifier(max_depth=8, gamma=0.3, subsample=0.75, min_child_weight=4, n_estimators=200, colsample_bytree = 0.75, objective='binary:logistic') 
    clf_xgb.fit(X_train, y_train)
    y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
    y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

    train_auc = metrics.roc_auc_score(y_train, y_train_pred)
    print("train auc: ", train_auc)
    test_auc = metrics.roc_auc_score(y_test, y_test_pred)
    print("test auc: ", test_auc)
print(len(X_train))
print(len(X_test))

TRAIN: [16167 16168 16169 ... 64664 64665 64666] TEST: [    0     1     2 ... 16164 16165 16166]
train auc:  0.9311781668902859
test auc:  0.7913502843365555
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [16167 16168 16169 ... 32331 32332 32333]
train auc:  0.9288422102250014
test auc:  0.7861010854697739
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [32334 32335 32336 ... 48498 48499 48500]
train auc:  0.9270387301140994
test auc:  0.79334533386399
TRAIN: [    0     1     2 ... 48498 48499 48500] TEST: [48501 48502 48503 ... 64664 64665 64666]
train auc:  0.9278340513652251
test auc:  0.7875696441391277
48501
16166


In [22]:
# 9 no id
kf =KFold(n_splits=4)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf_xgb = XGBClassifier(max_depth=8, gamma=0.3, subsample=0.75, min_child_weight=4, n_estimators=150, colsample_bytree = 0.75, objective='binary:logistic') 
    clf_xgb.fit(X_train, y_train)
    y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
    y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

    train_auc = metrics.roc_auc_score(y_train, y_train_pred)
    print("train auc: ", train_auc)
    test_auc = metrics.roc_auc_score(y_test, y_test_pred)
    print("test auc: ", test_auc)
print(len(X_train))
print(len(X_test))

TRAIN: [16167 16168 16169 ... 64664 64665 64666] TEST: [    0     1     2 ... 16164 16165 16166]
train auc:  0.914906087637018
test auc:  0.7916165515329872
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [16167 16168 16169 ... 32331 32332 32333]
train auc:  0.9128350740437428
test auc:  0.7855574340462964
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [32334 32335 32336 ... 48498 48499 48500]
train auc:  0.9128857380799902
test auc:  0.7936860109959687
TRAIN: [    0     1     2 ... 48498 48499 48500] TEST: [48501 48502 48503 ... 64664 64665 64666]
train auc:  0.912302845431968
test auc:  0.7881983509407647
48501
16166


In [23]:
# 10 no id
kf =KFold(n_splits=4)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf_xgb = XGBClassifier(max_depth=8, gamma=0.35, subsample=0.75, min_child_weight=4, n_estimators=150, colsample_bytree = 0.75, objective='binary:logistic') 
    clf_xgb.fit(X_train, y_train)
    y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
    y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

    train_auc = metrics.roc_auc_score(y_train, y_train_pred)
    print("train auc: ", train_auc)
    test_auc = metrics.roc_auc_score(y_test, y_test_pred)
    print("test auc: ", test_auc)
print(len(X_train))
print(len(X_test))

TRAIN: [16167 16168 16169 ... 64664 64665 64666] TEST: [    0     1     2 ... 16164 16165 16166]
train auc:  0.9140087298411759
test auc:  0.7926639028812599
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [16167 16168 16169 ... 32331 32332 32333]
train auc:  0.9115236158868094
test auc:  0.7861132889083162
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [32334 32335 32336 ... 48498 48499 48500]
train auc:  0.9124878138647678
test auc:  0.7931581352000588
TRAIN: [    0     1     2 ... 48498 48499 48500] TEST: [48501 48502 48503 ... 64664 64665 64666]
train auc:  0.9137876996084209
test auc:  0.7869694816394343
48501
16166


In [24]:
# 11 no id possible????
kf =KFold(n_splits=4)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf_xgb = XGBClassifier(max_depth=8, gamma=0.3, subsample=0.75, min_child_weight=4, n_estimators=170, colsample_bytree = 0.75, objective='binary:logistic') 
    clf_xgb.fit(X_train, y_train)
    y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
    y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

    train_auc = metrics.roc_auc_score(y_train, y_train_pred)
    print("train auc: ", train_auc)
    test_auc = metrics.roc_auc_score(y_test, y_test_pred)
    print("test auc: ", test_auc)
print(len(X_train))
print(len(X_test))

TRAIN: [16167 16168 16169 ... 64664 64665 64666] TEST: [    0     1     2 ... 16164 16165 16166]
train auc:  0.9225995553986317
test auc:  0.7917481581146186
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [16167 16168 16169 ... 32331 32332 32333]
train auc:  0.9206035086666997
test auc:  0.7861148335178887
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [32334 32335 32336 ... 48498 48499 48500]
train auc:  0.9188021287326771
test auc:  0.7935699782696521
TRAIN: [    0     1     2 ... 48498 48499 48500] TEST: [48501 48502 48503 ... 64664 64665 64666]
train auc:  0.9184305212642316
test auc:  0.7879490013748757
48501
16166


In [10]:
clf_xgb = XGBClassifier(max_depth=8, gamma=0.3, subsample=0.75, min_child_weight=4, n_estimators=170, colsample_bytree = 0.75, objective='binary:logistic') 
clf_xgb.fit(X, Y)
# y_test_pred_prob = clf_xgb.predict_proba(X_t)[:, 1]
# pd.DataFrame(y_test_pred_prob).to_csv("xgboost_1246.csv")

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.75, gamma=0.3, learning_rate=0.1,
       max_delta_step=0, max_depth=8, min_child_weight=4, missing=None,
       n_estimators=170, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.75)

In [11]:
test12_path = "test_2012.csv"
test12_data = read_file(test12_path)
X_test12 = test12_data
X_test12 = X_test12.drop('id', axis=1)
X_test12.drop(same_resp_feature, axis=1, inplace=True)
for feature in X_test12.columns:
    X_test12[feature] = X_test12[feature].apply(lambda x: np.NaN if (x == -1) else x)
X_t12 = X_test12.values
print(X_t12.shape)
y_test12_pred_prob = clf_xgb.predict_proba(X_t12)[:, 1]
pd.DataFrame(y_test12_pred_prob).to_csv("xgboost12_1246.csv")

(82820, 366)


In [12]:
# 14 no id
kf =KFold(n_splits=4)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf_xgb = XGBClassifier(max_depth=8, gamma=0.3, subsample=0.75, min_child_weight=4, n_estimators=160, colsample_bytree = 0.75, objective='binary:logistic') 
    clf_xgb.fit(X_train, y_train)
    y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
    y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

    train_auc = metrics.roc_auc_score(y_train, y_train_pred)
    print("train auc: ", train_auc)
    test_auc = metrics.roc_auc_score(y_test, y_test_pred)
    print("test auc: ", test_auc)
print(len(X_train))
print(len(X_test))

TRAIN: [16167 16168 16169 ... 64664 64665 64666] TEST: [    0     1     2 ... 16164 16165 16166]
train auc:  0.919218624768096
test auc:  0.7918286450351046
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [16167 16168 16169 ... 32331 32332 32333]
train auc:  0.9166384943645579
test auc:  0.7859215936540973
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [32334 32335 32336 ... 48498 48499 48500]
train auc:  0.9152395237343512
test auc:  0.7938879215600343
TRAIN: [    0     1     2 ... 48498 48499 48500] TEST: [48501 48502 48503 ... 64664 64665 64666]
train auc:  0.915286721783791
test auc:  0.7880224745070197
48501
16166


In [13]:
# 15 no id
kf =KFold(n_splits=4)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf_xgb = XGBClassifier(max_depth=8, gamma=0.3, subsample=0.8, min_child_weight=4, n_estimators=130, colsample_bytree = 0.8, objective='binary:logistic') 
    clf_xgb.fit(X_train, y_train)
    y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
    y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

    train_auc = metrics.roc_auc_score(y_train, y_train_pred)
    print("train auc: ", train_auc)
    test_auc = metrics.roc_auc_score(y_test, y_test_pred)
    print("test auc: ", test_auc)
print(len(X_train))
print(len(X_test))

TRAIN: [16167 16168 16169 ... 64664 64665 64666] TEST: [    0     1     2 ... 16164 16165 16166]
train auc:  0.9096913198044378
test auc:  0.7907391136459542
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [16167 16168 16169 ... 32331 32332 32333]
train auc:  0.9069782080247503
test auc:  0.7863752178666688
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [32334 32335 32336 ... 48498 48499 48500]
train auc:  0.9060436413223921
test auc:  0.7937588293934352
TRAIN: [    0     1     2 ... 48498 48499 48500] TEST: [48501 48502 48503 ... 64664 64665 64666]
train auc:  0.9079346916202776
test auc:  0.7885266945124495
48501
16166


In [14]:
clf_xgb = XGBClassifier(max_depth=8, gamma=0.3, subsample=0.8, min_child_weight=4, n_estimators=130, colsample_bytree = 0.8, objective='binary:logistic') 
clf_xgb.fit(X, Y)
y_test_pred_prob = clf_xgb.predict_proba(X_t)[:, 1]
pd.DataFrame(y_test_pred_prob).to_csv("xgboost_1338.csv")

In [10]:
# 11 no id no nan
kf =KFold(n_splits=4)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf_xgb = XGBClassifier(max_depth=8, gamma=0.3, subsample=0.75, min_child_weight=4, n_estimators=170, colsample_bytree = 0.75, objective='binary:logistic') 
    clf_xgb.fit(X_train, y_train)
    y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
    y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

    train_auc = metrics.roc_auc_score(y_train, y_train_pred)
    print("train auc: ", train_auc)
    test_auc = metrics.roc_auc_score(y_test, y_test_pred)
    print("test auc: ", test_auc)
print(len(X_train))
print(len(X_test))

TRAIN: [16167 16168 16169 ... 64664 64665 64666] TEST: [    0     1     2 ... 16164 16165 16166]
train auc:  0.9226867376729919
test auc:  0.7938282176322643
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [16167 16168 16169 ... 32331 32332 32333]
train auc:  0.9214900226025233
test auc:  0.7862612543219915
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [32334 32335 32336 ... 48498 48499 48500]
train auc:  0.9212164336606532
test auc:  0.7945869321071346
TRAIN: [    0     1     2 ... 48498 48499 48500] TEST: [48501 48502 48503 ... 64664 64665 64666]
train auc:  0.9187792363790558
test auc:  0.7894400231283052
48501
16166


In [25]:
# 12 no id
kf =KFold(n_splits=4)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf_xgb = XGBClassifier(max_depth=8, gamma=0.35, subsample=0.8, min_child_weight=4, n_estimators=150, colsample_bytree = 0.8, objective='binary:logistic') 
    clf_xgb.fit(X_train, y_train)
    y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
    y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

    train_auc = metrics.roc_auc_score(y_train, y_train_pred)
    print("train auc: ", train_auc)
    test_auc = metrics.roc_auc_score(y_test, y_test_pred)
    print("test auc: ", test_auc)
print(len(X_train))
print(len(X_test))

TRAIN: [16167 16168 16169 ... 64664 64665 64666] TEST: [    0     1     2 ... 16164 16165 16166]
train auc:  0.9179815137514167
test auc:  0.792558619638493
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [16167 16168 16169 ... 32331 32332 32333]
train auc:  0.9158856839365896
test auc:  0.7854367294571939
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [32334 32335 32336 ... 48498 48499 48500]
train auc:  0.9119556548758794
test auc:  0.7935975493119056
TRAIN: [    0     1     2 ... 48498 48499 48500] TEST: [48501 48502 48503 ... 64664 64665 64666]
train auc:  0.9135309140430259
test auc:  0.7896803988010008
48501
16166


In [26]:
# 13 no id
kf =KFold(n_splits=4)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf_xgb = XGBClassifier(max_depth=8, gamma=0.35, subsample=0.8, min_child_weight=4, n_estimators=170, colsample_bytree = 0.8, objective='binary:logistic') 
    clf_xgb.fit(X_train, y_train)
    y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
    y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

    train_auc = metrics.roc_auc_score(y_train, y_train_pred)
    print("train auc: ", train_auc)
    test_auc = metrics.roc_auc_score(y_test, y_test_pred)
    print("test auc: ", test_auc)
print(len(X_train))
print(len(X_test))

TRAIN: [16167 16168 16169 ... 64664 64665 64666] TEST: [    0     1     2 ... 16164 16165 16166]
train auc:  0.9236277887060527
test auc:  0.7924863543370992
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [16167 16168 16169 ... 32331 32332 32333]
train auc:  0.9217448735131841
test auc:  0.7854946676599632
TRAIN: [    0     1     2 ... 64664 64665 64666] TEST: [32334 32335 32336 ... 48498 48499 48500]
train auc:  0.9193726091690855
test auc:  0.7934682468322344
TRAIN: [    0     1     2 ... 48498 48499 48500] TEST: [48501 48502 48503 ... 64664 64665 64666]
train auc:  0.9200754390913893
test auc:  0.7895565551173157
48501
16166


In [12]:
test_size = 0.3
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=10)
clf_xgb = XGBClassifier(max_depth=9, gamma=0.25, subsample=0.7, min_child_weight=2, n_estimators=200, objective='binary:logistic', scale_pos_weight=3) 
clf_xgb.fit(X_train, y_train)
y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

train_auc = metrics.roc_auc_score(y_train, y_train_pred)
print("train auc: ", train_auc)
test_auc = metrics.roc_auc_score(y_test, y_test_pred)
print("test auc: ", test_auc)

train auc:  0.9800306961111053
test auc:  0.785634237348986


In [15]:
y_test_pred_prob = clf_xgb.predict_proba(X_t)[:, 1]
pd.DataFrame(y_test_pred_prob).to_csv("xgb_test_po2_2008.csv")

In [ ]:
test_size = 0.3
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=10)
clf_xgb = XGBClassifier(max_depth=9, gamma=0.2, subsample=0.8, min_child_weight=2, n_estimators=50, objective='binary:logistic') 
clf_xgb.fit(X_train, y_train)
y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

train_auc = metrics.roc_auc_score(y_train, y_train_pred)
print("train auc: ", train_auc)
test_auc = metrics.roc_auc_score(y_test, y_test_pred)
print("test auc: ", test_auc)